# Time Mixins

> Time datasets.

In [ ]:
#| default_exp time.mixs

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np, pandas as pd

import torch, torch.nn as nn, pytorch_lightning as pl
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

In [ ]:
#| export
from dataclasses import dataclass, field, KW_ONLY
from beartype.typing import Optional, Union, Iterable

from iza.static import TIME, SERIES
from iza.utils import Slice

In [ ]:
#| export
from littyping.core import (Device)

from litds.abc.dfdm.base import set_dataset, BaseDataFrameDataModule
from litds.abc.dfds.base import BaseDataFrameDataset
from litds.types import (SequenceWithLength, SequencesWithLengths)
from litds.mocks.time import MockTimeSeries

In [ ]:
#| eval: False
df = MockTimeSeries(set_index=True).df
df.head()

,time,feature_0,feature_1,feature_2
series,,,,
0,0,4,8,0
0,1,0,0,0
0,2,0,2,2
0,3,6,0,4
0,4,7,4,3


## TimeDatasetMixin

In [ ]:
#| export
@dataclass
class TimeDatasetMixin(BaseDataFrameDataset): 
    time_key: str = TIME

    _: KW_ONLY = field(default=None, init=False)
    include_time: Optional[bool] = False
    device: Optional[Device] = None
    
    # t_min: int = field(default=None, init=False)
    # t_max: int = field(default=None, init=False)
    # t_uni: List[int] = field(default=None, init=False)
    # t_idxs: Dict[int, Iterable] = field(default=None, init=False)

    def __post_init__(self):
        super().__post_init__()
        time_cats = self.df[self.time_key].astype('category')

        unique_timepoints = sorted(time_cats.cat.codes.unique())
        
        self.t_uni = unique_timepoints
        self.t_min = np.min(unique_timepoints)
        self.t_max = np.max(unique_timepoints)

        self.t_idxs = {
            t_val: (time_cats.cat.codes == t_val)
            for t_val in unique_timepoints
        }

    def __len__(self):
        # NOTE: we divide by the number of timepoints we have
        # even though it is not gaurenteed that we have an uniform number
        # of samples per time point

        # NOTE: this will impact the DataLoader later on
        # as the batch size will be the min(batch_size, len(self))
        return int(len(self.df) / len(self.t_uni))
        return len(self.t_uni)

    def getone(self):
        df_tmp = self.df 
        if not self.include_time:
            df_tmp = df_tmp.drop(columns=self.time_key, errors='ignore')

        one_sample = np.vstack(tuple([
            df_tmp[t_idx].sample(1) for t_idx in self.t_idxs.values()
        ]))
        timepoints = self.t_uni

        one_sample = torch.Tensor(one_sample)
        timepoints = torch.Tensor(timepoints)#.reshape(1, -1)

        if self.device is not None:
            one_sample = one_sample.to(self.device)
            timepoints = timepoints.to(self.device)

        return one_sample, timepoints
    
    def getmany(self, idx:int):        
        data = [self.getone() for _ in idx]
        seqs, time = zip(*data)
        # time = torch.tensor(time)
        seqs = torch.stack(seqs)
        time = torch.stack(time)
        return seqs, time

    
    def sample(self, t: int, n: int = 1, replace: bool = False):
        idx = self.t_idxs[t]
        return self.df[idx].sample(n=n, replace=replace)

    def __getitem__(self, idx):

        if isinstance(idx, slice):
            src = Slice(idx)
            arr = src.astype(list)
            idx = arr

        if isinstance(idx, Iterable):
            return self.getmany(idx)
        
        one_sample, timepoints = self.getone()
        return one_sample, timepoints

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()